In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from altaipony.lcio import from_path


/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/importlib/_bootstrap_external.py:426: ImportWarning: Not importing directory /home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/bokeh/resources.py:107: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/bokeh/_sri.json' mode='r' encoding='UTF-8'>
  _SRI_HASHES = json.load(open(join(ROOT_DIR, "_sri.json")))
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [18]:
df.shape

(2412, 17)

In [3]:
df = pd.read_csv("../k2scoc/results/tables/lost_flares.csv")#improved_flares_vetted
#df = df[(df.real == 1) & (df.note.isnull())]
df.head()

,istart,istop,cstart,cstop,tstart,tstop,ed_rec,ed_rec_err,ampl_rec,dtp_tot,C,EPIC,real,note,double_peak,complex,multiperiod
0,884,889,104666,104671,2247.639200,2247.741360,461.459128,6.446409,0.067284,3282,4,210667985,0,NaN,NaN,NaN,NaN
1,988,996,104774,104782,2249.845856,2250.009312,715.923723,19.962096,0.070269,3282,4,210667985,0,NaN,NaN,NaN,NaN
2,1078,1084,104868,104874,2251.766460,2251.889052,551.945640,27.179503,0.064460,3282,4,210667985,0,NaN,NaN,NaN,NaN
3,1126,1132,104919,104925,2252.808488,2252.931079,544.275192,41.803230,0.061412,3282,4,210667985,0,NaN,NaN,NaN,NaN
4,1148,1154,104942,104948,2253.278422,2253.401013,476.692257,9.051666,0.056668,3282,4,210667985,0,NaN,NaN,NaN,NaN


In [31]:
df = df.merge(clus[["C","EPIC","cluster"]], how="left", on=["C","EPIC"])

In [4]:
luminosities = pd.DataFrame()
for cl in ["ngc2682","ngc6774","praesepe","pleiades","hyades"]:
    lum = pd.read_csv(f"../StellarParameters/luminosities/{cl}_luminosities.csv")
    lum = lum[["EPIC","Lum_Kepler", "e_Lum_Kepler", "Teff_median","Teff_std", "todrop"]]
    lum.EPIC = lum.EPIC.astype(int)
    lum = lum.drop_duplicates()
    luminosities = luminosities.append(lum)

#df = df[(df.real == 1) & (df.note.isnull())]
print(luminosities.shape, luminosities.drop_duplicates().shape)
luminosities.head()

(3223, 6) (3223, 6)


,EPIC,Lum_Kepler,e_Lum_Kepler,Teff_median,Teff_std,todrop
0,211389428,1.952370e+31,3.641215e+30,4065.686917,86.142585,NaN
1,211389537,6.497467e+31,8.005714e+30,4859.417724,114.038717,g-z outlier
2,211389716,3.876991e+32,1.050844e+32,5995.022873,278.839086,NaN
3,211389931,2.112355e+31,3.856409e+30,4117.019702,121.777132,NaN
4,211390158,5.477966e+32,1.037648e+32,6271.767690,250.289954,NaN


## What are the lightcurves without flares?

take full flares table 

In [5]:
clus = pd.read_csv("lost_lcs.sh",delimiter=" ",#joblistonlyk2sc
                 names = ["python","call","EPIC","channel","C","cluster"
                         ])

In [8]:
df1.shape, df2.shape

((3790, 2), (940, 2))

In [9]:
df1 = clus[["EPIC","C"]]
df2 = df[["EPIC","C"]].drop_duplicates()

df3 = pd.concat([df1,df2]).drop_duplicates(keep=False)
df3.shape
df3.head()

,EPIC,C
0,211389428,5
1,211389428,16
2,211389537,16
4,211389716,16
5,211389716,18


In [10]:
for label, r in df3.iterrows():
    EPIC, C = int(r.EPIC), int(r.C)
    try:
        flc = from_path(f"../k2scoc/pony_k2sc_k2_llc_{EPIC}-c{C:02d}_kepler_v2_lc.fits",
                            mode="AltaiPony", mission="K2")
        flc = refine_detrended_flux_err(flc)

        df3.loc[(df3.EPIC == EPIC) & (df3.C == C),"dtp_tot"] = flc.flux.shape[0]
    except:
        df3.loc[(df3.EPIC == EPIC) & (df3.C == C),"dtp_tot"] = 0.

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211389428-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1667: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/home/ekaterina/Documents/001_science/flares-in-clusters-with-k2-ii/VettingFlares/error.py:42: RuntimeWarning: invalid value encountered in greater
  filtered[flcd.detrended_flux - flcd.it_med > mask_pos_outliers_sigma * flcd.detrended_flux_err] = np.nan
/home/ekaterina/Documents/001_science/flares-in-clusters-with-k2-ii/VettingFlare

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211397127-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211397319-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211397558-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211411112-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211412877-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211413212-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211420394-c16_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211420394-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211420952-c16_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211435897-c16_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211435897-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211435747-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

...

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_212001830-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_212016898-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211801146-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211790121-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211897200-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211727975-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211990578-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_212138965-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_212018903-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211936457-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211966629-c16_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211966629-c18_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211946907-c16_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211907293-c05_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipony/lcio.py:139: ResourceWarning: unclosed file <_io.FileIO name='../k2scoc/pony_k2sc_k2_llc_211907293-c16_kepler_v2_lc.fits' mode='rb' closefd=True>
  return _from_path_AltaiPony(path)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/altaipony-0.0.1-py3.6.egg/altaipon

In [11]:
df3b = df3.merge(clus[["C","EPIC","cluster"]], how="left", on=["C","EPIC"])

In [33]:
dffinal = pd.concat([df,df3b])
dffinal = dffinal[dffinal.dtp_tot > 0]
dffinal.head(40)

,istart,istop,cstart,cstop,tstart,tstop,ed_rec,ed_rec_err,ampl_rec,dtp_tot,C,EPIC,real,note,double_peak,complex,multiperiod,cluster
0,884.0,889.0,104666.0,104671.0,2247.639200,2247.741360,461.459128,6.446409,0.067284,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
1,988.0,996.0,104774.0,104782.0,2249.845856,2250.009312,715.923723,19.962096,0.070269,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
2,1078.0,1084.0,104868.0,104874.0,2251.766460,2251.889052,551.945640,27.179503,0.064460,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
3,1126.0,1132.0,104919.0,104925.0,2252.808488,2252.931079,544.275192,41.803230,0.061412,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
4,1148.0,1154.0,104942.0,104948.0,2253.278422,2253.401013,476.692257,9.051666,0.056668,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
5,1174.0,1180.0,104969.0,104975.0,2253.830083,2253.952675,532.773203,28.574259,0.057831,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
6,1228.0,1231.0,105027.0,105030.0,2255.015132,2255.076428,193.543524,9.880879,0.042051,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
7,1318.0,1324.0,105122.0,105128.0,2256.956158,2257.078749,369.852001,22.461565,0.037232,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
8,1517.0,1520.0,105328.0,105331.0,2261.165108,2261.226403,57.208511,6.449465,0.012825,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
9,1669.0,1673.0,105484.0,105488.0,2264.352459,2264.434186,29.962509,2.243580,0.005281,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades


In [34]:
dffinal.shape

(3091, 18)

In [35]:
dffinal

,istart,istop,cstart,cstop,tstart,tstop,ed_rec,ed_rec_err,ampl_rec,dtp_tot,C,EPIC,real,note,double_peak,complex,multiperiod,cluster
0,884.0,889.0,104666.0,104671.0,2247.639200,2247.741360,461.459128,6.446409,0.067284,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
1,988.0,996.0,104774.0,104782.0,2249.845856,2250.009312,715.923723,19.962096,0.070269,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
2,1078.0,1084.0,104868.0,104874.0,2251.766460,2251.889052,551.945640,27.179503,0.064460,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
3,1126.0,1132.0,104919.0,104925.0,2252.808488,2252.931079,544.275192,41.803230,0.061412,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
4,1148.0,1154.0,104942.0,104948.0,2253.278422,2253.401013,476.692257,9.051666,0.056668,3282.0,4,210667985,0.0,NaN,NaN,NaN,NaN,hyades
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2371.0,18,212092636,NaN,NaN,NaN,NaN,NaN,praesepe
2823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3446.0,5,211945852,NaN,NaN,NaN,NaN,NaN,praesepe
2824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2371.0,18,211945852,NaN,NaN,NaN,NaN,NaN,praesepe
2833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3446.0,5,212011328,NaN,NaN,NaN,NaN,NaN,praesepe


## Merge with luminosities

In [40]:
dflum = dffinal.merge(luminosities,how="left", on="EPIC")
dflum.head()

,istart,istop,cstart,cstop,tstart,tstop,ed_rec,ed_rec_err,ampl_rec,dtp_tot,...,note,double_peak,complex,multiperiod,cluster,Lum_Kepler,e_Lum_Kepler,Teff_median,Teff_std,todrop
0,884.0,889.0,104666.0,104671.0,2247.639200,2247.741360,461.459128,6.446409,0.067284,3282.0,...,NaN,NaN,NaN,NaN,hyades,2.517847e+31,4.536129e+30,4191.07,175.0,NaN
1,988.0,996.0,104774.0,104782.0,2249.845856,2250.009312,715.923723,19.962096,0.070269,3282.0,...,NaN,NaN,NaN,NaN,hyades,2.517847e+31,4.536129e+30,4191.07,175.0,NaN
2,1078.0,1084.0,104868.0,104874.0,2251.766460,2251.889052,551.945640,27.179503,0.064460,3282.0,...,NaN,NaN,NaN,NaN,hyades,2.517847e+31,4.536129e+30,4191.07,175.0,NaN
3,1126.0,1132.0,104919.0,104925.0,2252.808488,2252.931079,544.275192,41.803230,0.061412,3282.0,...,NaN,NaN,NaN,NaN,hyades,2.517847e+31,4.536129e+30,4191.07,175.0,NaN
4,1148.0,1154.0,104942.0,104948.0,2253.278422,2253.401013,476.692257,9.051666,0.056668,3282.0,...,NaN,NaN,NaN,NaN,hyades,2.517847e+31,4.536129e+30,4191.07,175.0,NaN


In [41]:
dflum.shape[0], df.shape[0] + df3b.shape[0]

(3091, 5262)

In [42]:
dflum.to_csv("../k2scoc/results/tables/lost_table.csv",index=False)